# JWST NIRSpec Observation Visualization Tool (NOVT): Aperture and Visibility Tools
-----------------------------------------------------------------------------------
**Author**: Melanie Clarke (mclarke@stsci.edu) | **Latest update**: May 17, 2023.
<a class="anchor" id="title"></a>

## Notebook Goals
Use NOVT tools to determine NIRSpec and NIRCam apertures and visibility for
a specific target within a range of dates.


## Table of contents
1. [Introduction](#intro)
2. [Determine visibility ranges](#visibility)
3. [Set aperture locations](#apertures)
4. [Check target coverage](#targets)
5. [Save DS9 regions](#regions)

<a class="anchor" id="intro"></a>
## Introduction

The NOVT package contains tools intended to assist in visualizing and planning NIRSpec observations, especially for multi-object spectroscopy (MOS) observations that require NIRCam pre-imaging observations.

STScI plans to host a web application that uses NOVT tools to plot target visibility and display NIRSpec and NIRCam apertures over a FITS image.  This notebook demonstrates how to use the same tools to generate inputs that may be used programmatically, or with your preferred visualization tool.

To start, we import some necessary packages.

In [ ]:
import datetime

import numpy as np
import pandas as pd
from astropy import units
from astropy.coordinates import SkyCoord
from astropy.stats import circmean
from astropy.wcs import WCS

from jwst_novt import footprints as fp
from jwst_novt import timeline as tl

For this example, we will set our target field to a location near the center of M51 (NGC 5194).

In [ ]:
# right ascension and declination in degrees
# center is at 202.46959, 47.195187
ra = 202.4792
dec = 47.1902

<a class="anchor" id="visibility"></a>
## Determine visibility ranges

JWST has ranges of time for which a particular target is visible. Within those ranges, the position angle (PA) for an instrument is tightly constrained by date. We use the NOVT timeline function to derive periods of visibility and the PA for NIRSpec and NIRCam during those times.

Note that this function uses astroquery to find a current ephemeris for JWST, so an internet connection is required to run it.

By default, the timeline function reports visibility data for one year from the current date for both NIRSpec and NIRCam.  The output is a dataframe with time and angle data.

In [ ]:
timeline_data = tl.timeline(ra, dec)
timeline_data

Times for which the target is not visible are marked in the angle columns with NaN values.  Let's take the first week for which the target is visible.

In [ ]:
# find all visible periods
visible = ~np.isnan(timeline_data["V3PA"])

# take the first visible date as the start date
start_date = timeline_data.loc[visible, "Time"].iloc[0]

# take the first visible date more than a week out as the end date
one_week = start_date + datetime.timedelta(days=7)
in_range = visible & (timeline_data["Time"] > one_week)
end_date = timeline_data.loc[in_range, "Time"].iloc[0]

print(f"Visible date range: {start_date} - {end_date}")

We can get the PA values for NIRCam and NIRSpec within those dates from the timeline dataframe.

In [ ]:
date_range = timeline_data["Time"].between(start_date, end_date, inclusive="both")
pa = timeline_data.loc[date_range]
pa

We can get the average value in the date range with a circular mean.

In [ ]:
# Use min and max PA data for each instrument, where values are not NaN
nircam_pa = np.array([pa["NIRCAM_min_PA"], pa["NIRCAM_max_PA"]])
nircam_avg = circmean(nircam_pa[~np.isnan(nircam_pa)] * units.deg)

nirspec_pa = np.array([pa["NIRSPEC_min_PA"], pa["NIRSPEC_max_PA"]])
nirspec_avg = circmean(nirspec_pa[~np.isnan(nirspec_pa)] * units.deg)

# Wrap negative angles
nircam_avg_pa = (nircam_avg.value + 360) % 360
nirspec_avg_pa = (nirspec_avg.value + 360) % 360

print(f"Average NIRCam PA this week: {nircam_avg_pa:.2f} deg")
print(f"Average NIRSpec PA this week: {nirspec_avg_pa:.2f} deg")

<a class="anchor" id="apertures"></a>
## Set aperture locations

We can use the NOVT footprints module to project JWST instrument apertures onto the sky at a given target location and with a specified position angle.

Target location and PA are set separately for NIRSpec and NIRCam, to allow optimization of particular pointings in planning the NIRCam pre-imaging, and to explore a range of possible position angles set by the observation timing.

NOVT returns aperture regions as a set of <a href="https://astropy-regions.readthedocs.io/en/stable/">astropy regions</a> in sky coordinates. These can be used to determine if a particular target is contained within a specified region, or can be displayed as an overlay on a FITS image of the target.

By default, the NIRSpec footprint includes all the NIRSpec apertures (MSA, IFU, fixed slits). Here, we retrieve only the four MSA apertures.

In [ ]:
nirspec_regions = fp.nirspec_footprint(
    ra,
    dec,
    nirspec_avg_pa,
    include_center=False,
    apertures=["NRS_FULL_MSA1", "NRS_FULL_MSA2", "NRS_FULL_MSA3", "NRS_FULL_MSA4"],
)
nirspec_regions

The Short and Long channels for NIRCam have separate, overlapping footprints.

In [ ]:
nircam_long_regions = fp.nircam_long_footprint(
    ra, dec, nircam_avg_pa, include_center=False
)
nircam_long_regions

In [ ]:
nircam_short_regions = fp.nircam_short_footprint(
    ra, dec, nircam_avg_pa, include_center=False
)
nircam_short_regions

NIRCam regions can additionally be computed with a dither pattern applied and/or with offsets for a two-tile mosaic.

In [ ]:
print("Number of regions in NIRCam long field: ", len(nircam_long_regions))

# use a 3-point dither pattern
dither_pattern = "FULL3"
dither = fp.nircam_dither_footprint(
    ra,
    dec,
    nircam_avg_pa,
    channel="long",
    include_center=False,
    dither_pattern=dither_pattern,
)
print("Number of regions in dithered NIRCam long field: ", len(dither))

# use a 2-tile mosaic
mosaic_offset = (20, 100)  # arcsec
mosaic = fp.nircam_dither_footprint(
    ra,
    dec,
    nircam_avg_pa,
    channel="long",
    include_center=False,
    add_mosaic=True,
    mosaic_offset=mosaic_offset,
)
print("Number of regions in mosaicked NIRCam long field: ", len(mosaic))

# use the 3-point pattern with the 2-tile mosaic
dither_mosaic = fp.nircam_dither_footprint(
    ra,
    dec,
    nircam_avg_pa,
    channel="long",
    include_center=False,
    dither_pattern=dither_pattern,
    add_mosaic=True,
    mosaic_offset=mosaic_offset,
)
print(
    "Number of regions in dithered and mosaicked NIRCam long field: ",
    len(dither_mosaic),
)

<a class="anchor" id="targets"></a>
## Check target coverage

Instrument aperture regions can be used to check whether a particular source is covered by both the NIRSpec MSA apertures and the NIRCam apertures.

If you have a source catalog in .radec form, the NOVT tools can read it in and create regions identifying the specified sources. This file should have the form of a text file with two or three columns, where the first is RA in degrees, the second is Dec in degrees, and the optional third column flags sources as either primary ('P') or filler ('F').

If you want to read your own catalog in to a set of regions, use the jwst_novt.footprints.source_catalog function.

In [ ]:
?fp.source_catalog

Here, we define a small catalog inline for testing.

In [ ]:
catalog = pd.DataFrame(
    {
        "ra": [
            202.46875,
            202.46587,
            202.47322,
            202.46553,
            202.46538,
            202.47324,
            202.47409,
            202.47295,
            202.46466,
            202.46596,
        ],
        "dec": [
            47.19884,
            47.19392,
            47.19707,
            47.19569,
            47.1945,
            47.19746,
            47.19543,
            47.19178,
            47.19452,
            47.19864,
        ],
        "flag": ["P", "P", "F", "F", "F", "P", "F", "F", "P", "F"],
    }
)
catalog

We can use the catalog to test if our aperture regions are covering our most important targets.

Note that sky regions require a WCS for 'contains' calculations. This can be read from a FITS file if you have one from which your source catalog was derived. We'll make a simple one for testing.

In [ ]:
# make a simple WCS centered at our target location
wcs = WCS(
    {
        "CTYPE1": "RA---TAN",
        "CUNIT1": "deg",
        "CDELT1": -1 / 3600.0,
        "CRPIX1": 1,
        "CRVAL1": ra,
        "CTYPE2": "DEC--TAN",
        "CUNIT2": "deg",
        "CDELT2": 1 / 3600,
        "CRPIX2": 1,
        "CRVAL2": dec,
    }
)

In [ ]:
# loop over source type to check target coverage
for flag in ["Primary", "Filler"]:
    # get the sources from the catalog by flag
    source = catalog[catalog["flag"] == flag[0]]
    n_source = len(source)

    # make sky coordinates for all the sources in the catalog
    coord = SkyCoord(source["ra"], source["dec"], unit="deg")

    # find the number of sources contained in any of the 4 NIRSpec quadrants
    contained = np.full(n_source, False)
    for reg in nirspec_regions:
        contained |= reg.contains(coord, wcs)
    print(
        f"{np.sum(contained)} / {n_source} {flag} sources "
        f"contained in NIRSpec apertures"
    )

    # find the number of sources contained in any of the NIRCam apertures
    contained = np.full(n_source, False)
    for reg in dither_mosaic:
        contained |= reg.contains(coord, wcs)
    print(
        f"{np.sum(contained)} / {n_source} {flag} sources "
        f"contained in dither/mosaic NIRCam apertures\n"
    )

Some of our primary and filler targets are contained in the NIRSpec apertures at the target position and specified PA, some are not. All our targets are contained in the dithered and mosaicked NIRCam pre-imaging field.

<a class="anchor" id="regions"></a>
## Save regions

Regions produced by the NOVT functions can be saved to disk using the astropy regions <a href="https://astropy-regions.readthedocs.io/en/stable/region_io.html">I/O functionality</a>.

We can save our already-generated instrument footprints to disk.

In [ ]:
nirspec_regions.write("novt_nirspec_regions.reg", format="ds9", overwrite=True)
dither_mosaic.write("novt_nircam_regions.reg", format="ds9", overwrite=True)

We can also make point regions for our catalog sources and save them.

In [ ]:
primary, filler = fp.source_catalog(catalog)
primary.write("novt_primary_catalog.reg", format="ds9", overwrite=True)
filler.write("novt_filler_catalog.reg", format="ds9", overwrite=True)

These regions can be loaded in to DS9, for overlay on a FITS image of the target region if desired.

[Top of Page](#title)
